# MNIST classification using GPUs

In [ ]:
import os

enable_gpu = True
if enable_gpu:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [1]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [1]:
'''Trains and evaluate a simple MLP
on the Reuters newswire topic classification task.
'''
from __future__ import print_function

import numpy as np

import keras
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.datasets import mnist
from keras.layers import Dense, Dropout, Activation
from keras.layers.convolutional import *
from keras.layers.core import *
from keras.models import Sequential
from keras.optimizers import *

import time

######################
nb_classes = 10
epochs = 5
batch_size = 16

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)
input_shape = (img_rows, img_cols, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, nb_classes)
y_test = keras.utils.to_categorical(y_test, nb_classes)

######################



print('Building model...')
# Construct the model.
model = Sequential()
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
model.add(Dense(225))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

start = time.time()

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

end = time.time()

score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)

print('Test score:', score[0])
print('Test accuracy:', score[1])
print("Training time: {0} seconds, ({1} minutes)".format(end-start, (end-start)/60.0))

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Building model...


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(28, 28, 1..., padding="valid")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`


Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 28s 519us/step - loss: 0.1114 - acc: 0.9666 - val_loss: 0.0537 - val_acc: 0.9857
Epoch 2/5
54000/54000 [==============================] - 25s 466us/step - loss: 0.0398 - acc: 0.9873 - val_loss: 0.0422 - val_acc: 0.9880
Epoch 3/5
54000/54000 [==============================] - 25s 465us/step - loss: 0.0238 - acc: 0.9923 - val_loss: 0.0356 - val_acc: 0.9912
Epoch 4/5
54000/54000 [==============================] - 25s 462us/step - loss: 0.0157 - acc: 0.9947 - val_loss: 0.0396 - val_acc: 0.9895
Epoch 5/5
10000/10000 [==============================] - 2s 161us/step
Test score: 0.05729995979192204
Test accuracy: 0.986
Training time: 128.30005741119385 seconds, (2.138334290186564 minutes)
